In [1]:
!ls

sample_data


In [8]:
# upload the crendential 

from google.colab import files
uploaded = files.upload() 


Saving google_cloud_creds2.json to google_cloud_creds2.json


In [0]:
#!pip install google-cloud-vision

In [0]:
# python 3 



# gcloud 
from google.cloud import vision
from google.oauth2 import service_account
from google.protobuf.json_format import MessageToDict

# OP 
import pandas as pd 
import numpy as np 

#--------------------------------------------------
# config 
# save ur google cloud credentials below
credentials = service_account.Credentials.from_service_account_file('google_cloud_creds2.json')
client = vision.ImageAnnotatorClient(credentials=credentials)


#--------------------------------------------------
# help func 


"""

# https://cloud.google.com/vision/docs/other-features

1) Image properties : vision.enums.Feature.Type.IMAGE_PROPERTIES
2) Web entities : vision.enums.Feature.Type.WEB_DETECTION
3) Safe search :  vision.enums.Feature.Type.SAFE_SEARCH_DETECTION

"""



def call_google_image_api(url,type):
    response = client.annotate_image({
    'image': {'source': {'image_uri': url}},
    'features': [{'type': type}],})
    print ('response : ', response)
    response_dict = MessageToDict(response)
    return response_dict



def g_face_detection(url):
    response = client.annotate_image({
    'image': {'source': {'image_uri': url}},
    'features': [{'type': vision.enums.Feature.Type.FACE_DETECTION}],})
    print ('response : ', response)
    response_dict = MessageToDict(response)
    return response_dict


def g_image_property(url):
    response = client.annotate_image({
    'image': {'source': {'image_uri': url}},
    'features': [{'type': vision.enums.Feature.Type.IMAGE_PROPERTIES}],})
    print ('response : ', response)
    response_dict = MessageToDict(response)
    return response_dict
  
  
def g_web_detection(url):
    response = client.annotate_image({
    'image': {'source': {'image_uri': url}},
    'features': [{'type': vision.enums.Feature.Type.WEB_DETECTION}],})
    print ('response : ', response)
    response_dict = MessageToDict(response)
    return response_dict




def call_google_handwritten_api(uri):

    from google.cloud import vision_v1p3beta1 as vision
    credentials = service_account.Credentials.from_service_account_file('google_cloud_creds2.json')
    
    
    client = vision.ImageAnnotatorClient(credentials=credentials)
    image = vision.types.Image()
    image.source.image_uri = uri

    # Language hint codes for handwritten OCR:
    # en-t-i0-handwrit, mul-Latn-t-i0-handwrit
    # Note: Use only one language hint code per request for handwritten OCR.
    image_context = vision.types.ImageContext(
        language_hints=['en-t-i0-handwrit'])

    response = client.document_text_detection(image=image,
                                              image_context=image_context)

    print('Full Text: {}'.format(response.full_text_annotation.text))
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print('\nBlock confidence: {}\n'.format(block.confidence))

            for paragraph in block.paragraphs:
                print('Paragraph confidence: {}'.format(
                    paragraph.confidence))

                for word in paragraph.words:
                    word_text = ''.join([
                        symbol.text for symbol in word.symbols
                    ])
                    print('Word text: {} (confidence: {})'.format(
                        word_text, word.confidence))

                    for symbol in word.symbols:
                        print('\tSymbol: {} (confidence: {})'.format(
                            symbol.text, symbol.confidence))


#--------------------------------------------------


def get_web_entity(web_property_response):
  return [i for i in web_property_response['webDetection']['webEntities'] ]


In [0]:
df_10k_random = pd.read_csv('random_10K_image_urls.csv')

In [11]:
df_10k_random.head(3)

,url
0,https://assets-notonthehighstreet-com.s3.amazo...
1,https://assets-notonthehighstreet-com.s3.amazo...
2,https://assets-notonthehighstreet-com.s3.amazo...


In [17]:
# sample data 
df_10k_random_ = df_10k_random.head(10)

# query google vision api 
df_10k_random_['image_property'] = df_10k_random_['url'].apply(lambda x : g_image_property(x))
df_10k_random_['web_detection'] = df_10k_random_['url'].apply(lambda x : g_web_detection(x))
# extract web entity information 
df_10k_random_['web_entity'] = df_10k_random_['web_detection'].apply(lambda x : get_web_entity(x))


response :  image_properties_annotation {
  dominant_colors {
    colors {
      color {
        red: 166.0
        green: 162.0
        blue: 153.0
      }
      score: 0.34678566455841064
      pixel_fraction: 0.18145833909511566
    }
    colors {
      color {
        red: 64.0
        green: 96.0
        blue: 68.0
      }
      score: 0.03624964505434036
      pixel_fraction: 0.006527777761220932
    }
    colors {
      color {
        red: 171.0
        green: 163.0
        blue: 146.0
      }
      score: 0.15224076807498932
      pixel_fraction: 0.07777778059244156
    }
    colors {
      color {
        red: 186.0
        green: 185.0
        blue: 179.0
      }
      score: 0.07636457681655884
      pixel_fraction: 0.12479166686534882
    }
    colors {
      color {
        red: 121.0
        green: 120.0
        blue: 114.0
      }
      score: 0.04679257050156593
      pixel_fraction: 0.03187499940395355
    }
    colors {
      color {
        red: 86.0
        green: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


response :  web_detection {
  web_entities {
    entity_id: "/m/01h3n"
    score: 0.7855499982833862
    description: "Bee"
  }
  web_entities {
    entity_id: "/m/023j4r"
    score: 0.7027000188827515
    description: "Nail"
  }
  web_entities {
    entity_id: "/m/0zdksk1"
    score: 0.5324000120162964
    description: "Nail art"
  }
  web_entities {
    entity_id: "/t/24mypdx4svpvn"
    score: 0.4966999888420105
  }
  web_entities {
    entity_id: "/m/02csf"
    score: 0.4625000059604645
    description: "Drawing"
  }
  web_entities {
    entity_id: "/m/0204p6"
    score: 0.43299999833106995
    description: "Decal"
  }
  web_entities {
    entity_id: "/m/05b97g"
    score: 0.3301999866962433
    description: "Sticker"
  }
  web_entities {
    entity_id: "/t/2559cfx2jwyf6"
    score: 0.32350000739097595
  }
  web_entities {
    entity_id: "/m/0rzp91s"
    score: 0.3156999945640564
  }
  web_entities {
    entity_id: "/m/0fz0z"
    score: 0.3147999942302704
    description: "Honey bee

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [18]:
df_10k_random_.head(3)

,url,image_property,web_detection,web_entity
0,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998..."
1,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/03lnq3', 'score': 0.89781457..."
2,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/0cjkt', 'score': 0.700600028..."


In [0]:
# convert web_entity to df 
frame = pd.DataFrame()
list_ = []

#for i in range(5):
for i in range(len(df_10k_random_)):
  url_ = df_10k_random_.iloc[i]['url']
  df_web_entity = pd.DataFrame(df_10k_random_.iloc[i]['web_entity'])
  df_web_entity['url'] = url_
  # merge 
  df_10k_random_merge = pd.merge(df_10k_random_, df_web_entity,  how='inner', left_on=['url'], right_on=['url'])
  list_.append(df_10k_random_merge)
frame = pd.concat(list_)
frame = frame.reset_index()

In [20]:
frame

,index,url,image_property,web_detection,web_entity,description,entityId,score
0,0,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",Bee,/m/01h3n,0.785550
1,1,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",Nail,/m/023j4r,0.702700
2,2,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",Nail art,/m/0zdksk1,0.532400
3,3,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",NaN,/t/24mypdx4svpvn,0.496700
4,4,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",Drawing,/m/02csf,0.462500
5,5,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",Decal,/m/0204p6,0.433000
6,6,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",Sticker,/m/05b97g,0.330200
7,7,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",NaN,/t/2559cfx2jwyf6,0.323500
8,8,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",NaN,/m/0rzp91s,0.315700
9,9,https://assets-notonthehighstreet-com.s3.amazo...,{'imagePropertiesAnnotation': {'dominantColors...,{'webDetection': {'webEntities': [{'entityId':...,"[{'entityId': '/m/01h3n', 'score': 0.785549998...",Honey bee,/m/0fz0z,0.314800
